In karaoke, lyrics are presented on screen in rough synchronization with the appropriate portion of the song. However, unless manually done, the lyric guide (pointer/highlight over the appropriate word to be sung at a time) is absent. I attempt to automate this process.

The high level idea is to use a pre-trained network (RNN?) on speech transcription, then use a similar idea to an image heatmap in CNNs to find the portion of the audio input that contributed most to the chosen word. With these time markers, we now have a lyric guide.

Background: speech-to-text networks, CNN heatmaps

Formally, this is known as the "forced alignment problem"

## Apporach 1: Using GRAD-CAM on CNN's

#### For this, I will use end-to-end CNN's for ASR and apply the GRAD-CAM method described by Selvaraju et. al (2016) in [this paper](https://arxiv.org/abs/1610.02391). These approaches will follow a Connecitonist Temporal Classification (CTC) model.

### Approach 1a: I will implement [this paper](https://arxiv.org/pdf/1701.02720.pdf) by Zhang et. al (2017) for the CNN.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# TODO: Get CUDA working
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# Is this function really necessary?

def get_libri_speech_dataset(dataset_dir: str, dataset: str="train-clean-100") -> torch.utils.data.Dataset:

    """
    Function to download LibriSpeech dataset.

    Inputs: 
    dataset_dir -- Path to directory where dataset should be located/downloaded
    dataset -- Type of dataset desired. Options are "train-clean-100", "train-clean-360", "train-clean-500", "dev-clean", "dev-other", "test-clean", "test-other"
    dataset

    Output: torch.utils.data.Dataset of tuples with contents (waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id)
    """

    # can use either key or url for "url" parameter of dataset download function
    return torchaudio.datasets.LIBRISPEECH(dataset_dir, url=dataset, download=True)

In [68]:
char_map_str = """
 <SPACE> 1
 a 2
 b 3
 c 4
 d 5
 e 6
 f 7
 g 8
 h 9
 i 10
 j 11
 k 12
 l 13
 m 14
 n 15
 o 16
 p 17
 q 18
 r 19
 s 20
 t 21
 u 22
 v 23
 w 24
 x 25
 y 26
 z 27
 ' 28
 """

def create_char_map(char_map_str):
    char_map = {}
    for line in char_map_str.strip().split("\n"):
        c, num = line.split()
        char_map[c] = int(num)
    return char_map

char_map = create_char_map(char_map_str)

def text_to_target(text, char_map):
    target = []
    for c in text:
        if c == " ":
            target.append(char_map["<SPACE>"])
        else:
            target.append(char_map[c])
    return torch.Tensor(target)



In [69]:
def features_from_waveform(waveform):

    """
    Raw audio is transformed into 40-dimensional log mel-filter-bank (plus energy term) coefficients with deltas and delta-deltas, which reasults in 123 dimensional features. Each dimension is normalized to have zero mean and unit variance over the training set.

    Basically this is just MFCC but without taking DCT at the end, but for the sake of cleanliness, I'll stick with MFCC for now. Also, I don't know what they mean by "energy term" (aren't the coefficients already energy terms?) so I'm omitting that.
    """

    # Waveform has channel first dimension, gives shape [1, ...] which causes shape problems when stacking features
    data = waveform.squeeze(dim=0)

    # Grab desired features
    # Takes in audio of dimension (..., time) returns specgram_mel of size (..., n_mfcc, time) where n_mfcc defaults to 40
    mfcc_features = torchaudio.transforms.MFCC(log_mels=True)(data)
    deltas = torchaudio.functional.compute_deltas(mfcc_features)
    delta_deltas = torchaudio.functional.compute_deltas(deltas)

    # Stack features together
    input_features = torch.cat((mfcc_features, deltas, delta_deltas), 0)

    # Normalize features along time dimension
    # PyTorch built in Normalize transform does it to entire channels, we want along specific dimension of only channel, which we can do with LayerNorm Module

    input_features_normalized = nn.LayerNorm(input_features.shape[1], elementwise_affine=False)(input_features)

    # Old ways of experimenting with normalizing along dimensions. 
    
    # Unsqueeze method
    # input_features_normalized = (input_features - input_features.mean(dim=1).unsqueeze(dim=1)) / input_features.std(dim=1).unsqueeze(dim=1)

    # Transpose method
    # input_features_normalized = ((input_features.T - input_features.T.mean(0)) / input_features.T.std(0)).T

    # Numpy method: Using a useful function numpy has but torch does not. Creates small floating point differences, probably from conversion to torch tensor from numpy array.
    # input_features_normalized = torch.from_numpy(np.apply_along_axis(lambda x : (x - np.mean(x)) / np.std(x), 1, input_features))

    return input_features_normalized

def preprocess(dataset):

    """
    Preprocesses dataset

    1. Convert waveforms to input features, pad for consistent sizes
    2. TODO: Conver labels to something
    """

    inputs = [] # Features that are input to model
    targets = [] # Target transcripts, NOT output of model, used for CTC Loss
    # For these, CTCLoss expects them to be pre-padding
    input_lengths = [] # Time lengths of input features
    target_lengths = [] # Length of target transcripts

    # Each waveform has different lengths of time dimension, so we need to pad them. The built-in fn assumes trailing dimensions of all sequences are the same, so we have to transpose to pad the correct dimension since time dim (dim=1) is different, then transpose back after.

    # Current output of features_from_waveform have dim: 120 x time
    for waveform, _, transcript, _, _, _ in dataset:
        features = features_from_waveform(waveform).transpose(0, 1)
        inputs.append(features)
        input_lengths.append(features.shape[0])

        target = text_to_target(transcript.lower(), char_map)
        targets.append(target)
        target_lengths.append(len(target))

    # Returns dimensions: num_inputs (batch) x time x features, so transpose appropriately
    # Need to add back (unsqueeze) channel dimension
    # Side note: this will pad all samples within the same batch (not overall) to be the same dimensions
    # TODO: Think about how this affects the dimensions of conv -> fc layers
    inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True).unsqueeze(1).transpose(2, 3) 

    # This will pad with 0, which represents the blank, but this shouldn't be a problem since we're providing the target_length of the unpadded target
    targets = nn.utils.rnn.pad_sequence(targets, batch_first=True)

    return inputs, input_lengths, targets, target_lengths



In [70]:
# TODO: Dataloader stuff to handle batching datasets
# Want DataLoader to take in raw dataset, batch them, pass into collate_fn to perform all necessary transformations, and return output
# Output of form (feature_maps)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=hparams["batch_size"], shuffle=False, collate_fn=preprocess)



In [5]:
def maxout():
    # TODO: Implement this
    pass

class Conv_Maxout_Layer(nn.Module):

    def __init__(self, in_channels, out_channels, filter_dim=(3,5), pool=False, pool_dim=(3,1), dropout=0.3):
        super(Conv_Maxout_Layer, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=filter_dim)
        # Using Dropout module vs functional will automatically disable it during model.eval()
        self.dropout = nn.Dropout(p=dropout)
        self.pool = pool
        self.pool_dim = pool_dim

    def forward(self, x):
        x = self.conv(x)
        if self.pool:
            x = nn.MaxPool2d(kernel_size=self.pool_dim)(x)
        # maxout
        # doing this after pooling has faster runtime because less operations
        x = self.dropout(x)
        return x

class FC_Layer(nn.Module):

    def __init__(self, in_dim, out_dim, dropout=0.3):
        super(FC_Layer, self).__init__()

        self.fc = nn.Linear(in_dim, out_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.fc(x)
        # maxout
        x = self.dropout(x)
        return x

class ASR_1(nn.Module):

        """
        Unlike the other layers, the first convolutional layer is followed by a pooling layer, which is described in section 2. The pooling size is 3 × 1, which mean swe only pool over the frequency axis. The filter size is 3 × 5 across the layers. The model has 128 feature maps in the first four convolutional layers and 256 feature maps in the remaining six convolutional layers. Each fully-connected layer has 1024 units. Maxout with 2 piece-wise linear functions is used as the activation function.
        """ 

        def __init__(self, in_dim, out_dim):
            super(ASR_1, self).__init__()
            self.in_dim = in_dim 
            self.cnn_layers = nn.Sequential(
                            Conv_Maxout_Layer(self.in_dim, 128, pool=True, dropout=0),
                            Conv_Maxout_Layer(128, 128),
                            Conv_Maxout_Layer(128, 128),
                            Conv_Maxout_Layer(128, 128),
                            Conv_Maxout_Layer(128, 256),
                            Conv_Maxout_Layer(256, 256),
                            Conv_Maxout_Layer(256, 256),
                            Conv_Maxout_Layer(256, 256),
                            Conv_Maxout_Layer(256, 256),
                            Conv_Maxout_Layer(256, 256),
            )
            # TODO: Figure out dimension from last conv to first fc
            self.fc_layers = nn.Sequential(
                            FC_Layer(1, 1024),
                            FC_Layer(1024, 1024),
                            FC_Layer(1024, 1024),
                            FC_Layer(1024, out_dim, dropout=0)
            )

            # TODO: need to add last layer for character classes (+ blank for CTC?)


        def forward(self, x):
            x = self.cnn_layers(x)
            # TODO: transfomration from cnn dimensions to fc dimensions
            x = self.fc_layers(x)
            # TODO: Add log softmax at end for CTCLoss to work:  torch.nn.functional.log_softmax()
            return x


In [6]:
# Old way of initializing weights by iterating through sub-modules rather than parameters directly
# def weights_init_unif(m, a, b):
#     if type(m) is nn.Linear or type(m) is nn.Conv2d:
#         nn.init.uniform_(m.weight.data, a=a, b=b)
#         nn.init.uniform_(m.bias.data, a=a, b=b)

def weights_init_unif(module, a, b):
    for p in module.parameters():
        nn.init.uniform_(p.data, a=a, b=b)

In [92]:
PATH_TO_DATASET_DIR = "/mnt/d/Datasets/"

# def main():

hparams = {
    "ADAM_lr": 10e-4,
    "batch_size": 20,
    "SGD_lr": 10e-5,
    "SGD_l2_penalty": 1e-5,
    "weights_init_a": -0.05,
    "weights_init_b": 0.05,
    "epochs": 10
}

train_dataset = get_libri_speech_dataset(PATH_TO_DATASET_DIR)
dev_dataset = get_libri_speech_dataset(PATH_TO_DATASET_DIR, "dev-clean")
test_dataset = get_libri_speech_dataset(PATH_TO_DATASET_DIR, "test-clean")

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=hparams["batch_size"], shuffle=True, collate_fn=preprocess)

# dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=hparams["batch_size"], shuffle=True, collate_fn=preprocess)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=hparams["batch_size"], shuffle=False, collate_fn=preprocess)

# 1 channel input from feature spectrogram, 29 dim output from char_map + blank for CTC
net = ASR_1(1, 29)
# net.apply(lambda x: weights_init_unif(x, hparams["weights_init_a"], hparams["weights_init_b"]))
weights_init_unif(net, hparams["weights_init_a"], hparams["weights_init_b"])

# ADAM loss w/ lr=10e-4, batch size 20, initial weights initialized uniformly from [-0.05, 0.05], dropout w/ p=0.3 used in all layers except in and out
# for fine tuning: SGD w/ lr 10e-5, l2 penalty w/ coeff=1e-5

criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=hparams["ADAM_lr"])
finetune_optimizer = torch.optim.SGD(net.parameters(), lr=hparams["SGD_lr"], weight_decay=hparams["SGD_l2_penalty"])

# for epoch in range(hparams["epochs"]):
#     train(net, train_loader, criterion, optimizer)
    
# TODO: Where/when to do dev set?

# test(net, test_loader, criterion)

In [89]:
def train(model, train_dataset, criterion, optimizer):
    model.train()
    for inputs, input_lengths, targets, target_lengths in train_dataset:

        optimizer.zero_grad()

        # check what shape this is
        output = model(inputs)

        loss = criterion(output, targets, input_lengths, target_lengths)
        loss.backwards()

        optimizer.step() 

def test(model, test_dataset, criterion):
    model.eval()
    with torch.no_grad():
        for inputs, input_lengths, targets, target_lengths in test_dataset:

            output = model(inputs)

            loss = criterion(output, targets, input_lengths, target_lengths)
            
            # TODO: Decoding algo